<a href="https://colab.research.google.com/github/Knowsoeun/Fraudit-lecture/blob/main/%EC%9D%98%EB%A5%98%EC%A7%81%EB%AC%BC_%EB%B6%88%EB%9F%89%EA%B2%80%EC%B6%9C_%EC%9D%B4%EB%AF%B8%EC%A7%80_%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

실습 환경 준비

In [ ]:
!pip install opencv-python

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import cv2     #컴퓨터 비전 오픈소스 라이브러리
import os
import glob
import shutil
import random
import string
import numpy as np

In [ ]:
PATH_DEFECT = '/content/drive/MyDrive/AI스터디/dataset/Defect_images/Defect_images/'
PATH_MASK = '/content/drive/MyDrive/AI스터디/dataset/Mask_images/Mask_images/'
PATH_NODEFECT = '/content/drive/MyDrive/AI스터디/dataset/NODefect_images/NODefect_images/'

Dataset 불러오기

In [ ]:
random.seed(0)

defect_list = glob.glob(PATH_DEFECT + '*.png')
mask_list = glob.glob(PATH_MASK + '*.png')
pass_list = glob.glob(PATH_NODEFECT + '**/*.png')

# Match defect-mask pairs
new_defect_list = list()
new_mask_list = list()
for defect in defect_list:
    num = defect.split('/')[-1].split('_')[0]
    for mask in mask_list:
        num_mask = mask.split('/')[-1].split('_')[0]
        if num == num_mask:
            new_defect_list.append(defect)
            new_mask_list.append(mask)
            break
defect_list = new_defect_list
mask_list = new_mask_list

첫 발송 데이터 생성

In [ ]:
# The first dataset given
if os.path.exists('/content/drive/MyDrive/AI스터디/dataset/1') is False:
    os.mkdir('/content/drive/MyDrive/AI스터디/dataset/1')
for file_name in pass_list + defect_list:
    if random.randint(0, 9) < 2:
        barcode = ''.join(random.choices(string.ascii_letters + string.digits, k=16))
        shutil.copy2(file_name, '/content/drive/MyDrive/AI스터디/dataset/1/' + barcode + '.png')

두번째 데이터 생성

In [ ]:
# The second dataset
if os.path.exists('/content/drive/MyDrive/AI스터디/dataset/2') is False:
    os.mkdir('/content/drive/MyDrive/AI스터디/dataset/2')
if os.path.exists('/content/drive/MyDrive/AI스터디/dataset/2/OK') is False:
    os.mkdir('/content/drive/MyDrive/AI스터디/dataset/2/OK')
if os.path.exists('/content/drive/MyDrive/AI스터디/dataset/2/FAIL') is False:
    os.mkdir('/content/drive/MyDrive/AI스터디/dataset/2/FAIL')
idx = 0
for file_name in pass_list:
    img = cv2.imread(file_name)
    height, width, _ = img.shape
    step = height // 2

    for i in range(width // step):
        w = i * step
        if w < width - height and random.randint(0, 9) < 2:
            patch = img[:, w:w+height, :]
            cv2.imwrite('/content/drive/MyDrive/AI스터디/dataset/2/OK/%04d.png' % idx, patch)
            idx += 1 

patch_list = list()
for item in zip(defect_list, mask_list):
    defect, mask = item

    img_d = cv2.imread(defect)
    img_m = cv2.imread(mask)

    height, width, _ = img_d.shape
    step = height // 2
    for i in range(width // step):
        w = i * step
        if w < width - height:
            patch = img_d[:, w:w+height, :]
            patch_d = img_m[:, w:w+height, :]
            if patch_d.sum() > 0:
                patch_list.append(patch)

random.shuffle(patch_list)
patch_list_fraction = patch_list[:len(patch_list)//3]
for idx, patch in enumerate(patch_list_fraction):
    cv2.imwrite('/content/drive/MyDrive/AI스터디/dataset/2/FAIL/%04d.png' % idx, patch)

세번째 데이터 생성

In [ ]:
# The third dataset
if os.path.exists('/content/drive/MyDrive/AI스터디/dataset/3') is False:
    os.mkdir('/content/drive/MyDrive/AI스터디/dataset/3')
if os.path.exists('/content/drive/MyDrive/AI스터디/dataset/3/OK') is False:
    os.mkdir('/content/drive/MyDrive/AI스터디/dataset/3/OK')
if os.path.exists('/content/drive/MyDrive/AI스터디/dataset/3/FAIL') is False:
    os.mkdir('/content/drive/MyDrive/AI스터디/dataset/3/FAIL')
if os.path.exists('/content/drive/MyDrive/AI스터디/dataset/3/MASK') is False:
    os.mkdir('/content/drive/MyDrive/AI스터디/dataset/3/MASK')
idx = 0
for file_name in pass_list:
    img = cv2.imread(file_name)
    height, width, _ = img.shape
    step = height // 2

    for i in range(width // step):
        w = i * step
        if w < width - height and random.randint(0, 9) < 3:
            patch = img[:, w:w+height, :]
            cv2.imwrite('/content/drive/MyDrive/AI스터디/dataset/3/OK/%04d.png' % idx, patch)
            idx += 1 

patch_pair_list = list()
for item in zip(defect_list, mask_list):
    defect, mask = item

    img_d = cv2.imread(defect)
    img_m = cv2.imread(mask)

    height, width, _ = img_d.shape
    step = height // 2
    for i in range(width // step):
        w = i * step
        if w < width - height:
            patch = img_d[:, w:w+height, :]
            patch_d = img_m[:, w:w+height, :]

            if patch_d.sum() > 0:
                patch_pair_list.append((patch, patch_d))

random.shuffle(patch_pair_list)
for idx, pair in enumerate(patch_pair_list):
    patch, patch_d = pair
    cv2.imwrite('/content/drive/MyDrive/AI스터디/dataset/3/FAIL/%04d.png' % idx, patch)
    cv2.imwrite('/content/drive/MyDrive/AI스터디/dataset/3/MASK/%04d.png' % idx, patch_d)

실전 데이터 생성

In [ ]:
# The test dataset
if os.path.exists('/content/drive/MyDrive/AI스터디/data/') is False:
    os.mkdir('/content/drive/MyDrive/AI스터디/data/')
if os.path.exists('/content/drive/MyDrive/AI스터디/tfrecords/') is False:
    os.mkdir('/content/drive/MyDrive/AI스터디/tfrecords/')
if os.path.exists('/content/drive/MyDrive/AI스터디/model/') is False:
    os.mkdir('/content/drive/MyDrive/AI스터디/model/')
if os.path.exists('/content/drive/MyDrive/AI스터디/data/input_data') is False:
    os.mkdir('/content/drive/MyDrive/AI스터디/data/input_data')
if os.path.exists('/content/drive/MyDrive/AI스터디/data/output_csv') is False:
    os.mkdir('/content/drive/MyDrive/AI스터디/data/output_csv')
    
idx = 0
for file_name in pass_list:
    img = cv2.imread(file_name)
    height, width, _ = img.shape
    step = height // 2

    for i in range(width // step):
        w = i * step
        if w < width - height and random.randint(0, 9) < 5:
            patch = img[:, w:w+height, :]
            cv2.imwrite('/content/drive/MyDrive/AI스터디/data/input_data/ok_%04d.png' % idx, patch)
            idx += 1 

patch_pair_list = list()
for item in zip(defect_list, mask_list):
    defect, mask = item

    img_d = cv2.imread(defect)
    img_m = cv2.imread(mask)

    height, width, _ = img_d.shape
    step = height // 2
    for i in range(width // step):
        w = i * step
        if w < width - height:
            patch = img_d[:, w:w+height, :]
            patch_d = img_m[:, w:w+height, :]

            if patch_d.sum() > 0:
                patch_pair_list.append((patch, patch_d))

random.shuffle(patch_pair_list)
for idx, pair in enumerate(patch_pair_list):
    patch, patch_d = pair
    cv2.imwrite('/content/drive/MyDrive/AI스터디/data/input_data/fail_%04d.png' % idx, patch)

실행 가능성 확인하기

In [ ]:
import glob
import os
import tensorflow as tf
import cv2
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from tensorflow.keras.models import Sequential

하이퍼파라미터

In [ ]:
EPOCHS = 5  # 학습이 너무 느려서 5로 설정함

DATASET_PATH = '/content/drive/MyDrive/AI스터디/dataset/2/'
DATASET_OK_PATTERN = DATASET_PATH + 'OK/*.png'     # 정상
DATASET_FAIL_PATTERN = DATASET_PATH + 'FAIL/*.png'  # 불량

RESULT_SAVE_PATH = '/content/drive/MyDrive/AI스터디/results/'  # 알고리즘 결과

단순한 모델 설정

In [ ]:
def Model():
    return Sequential([Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 1)), #tensorflow 버전업으로 코드 변경
                       MaxPool2D(),
                       Conv2D(64, (3, 3), activation='relu'),
                       MaxPool2D(),
                       Conv2D(128, (3, 3), activation='relu'),
                       MaxPool2D(),
                       Conv2D(256, (3, 3), activation='relu'),
                       MaxPool2D(),
                       Flatten(),
                       Dense(1, activation='sigmoid')])

데이터셋 불러오기

In [ ]:
def preprocess(file_name):
    img = tf.io.read_file(file_name)
    img = tf.image.decode_png(img, channels=1) #tensorflow 버전업으로 코드 변경
    return tf.image.convert_image_dtype(img, tf.float32)

In [ ]:
ok_list = glob.glob(DATASET_OK_PATTERN)
ds_ok = tf.data.Dataset.list_files(ok_list)
ds_ok_label = tf.data.Dataset.from_tensor_slices([0] * len(ok_list))

ds_ok = ds_ok.map(preprocess)
ds_ok = tf.data.Dataset.zip((ds_ok, ds_ok_label))

fail_list = glob.glob(DATASET_FAIL_PATTERN)
ds_fail = tf.data.Dataset.list_files(fail_list)
ds_fail_label = tf.data.Dataset.from_tensor_slices([1] * len(fail_list))

ds_fail = ds_fail.map(preprocess)
ds_fail = tf.data.Dataset.zip((ds_fail, ds_fail_label))

ds = tf.data.Dataset.concatenate(ds_ok, ds_fail)

Train, Valid 데이터셋 나누기

In [ ]:
ds_size = len(ok_list) + len(fail_list)
train_size = int(ds_size * 0.7)

ds = ds.shuffle(ds_size)
ds_train = ds.take(train_size).shuffle(1024, reshuffle_each_iteration=True).batch(32)
ds_valid = ds.skip(train_size).batch(32)

모델 생성 및 학습

In [ ]:
model = Model()
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [91]:
model.fit(ds_train, validation_data=ds_valid, epochs=EPOCHS)

Epoch 1/5
21/21 [==============================] - 120s 5s/step - loss: 0.4538 - accuracy: 0.8805 - val_loss: 0.4225 - val_accuracy: 0.8732
Epoch 2/5
21/21 [==============================] - 105s 5s/step - loss: 0.3734 - accuracy: 0.8835 - val_loss: 0.4375 - val_accuracy: 0.8662
Epoch 3/5
21/21 [==============================] - 105s 5s/step - loss: 0.3818 - accuracy: 0.8759 - val_loss: 0.3600 - val_accuracy: 0.8768
Epoch 4/5
21/21 [==============================] - 105s 5s/step - loss: 0.3529 - accuracy: 0.8896 - val_loss: 0.3468 - val_accuracy: 0.8873
Epoch 5/5
21/21 [==============================] - 105s 5s/step - loss: 0.3617 - accuracy: 0.8820 - val_loss: 0.3457 - val_accuracy: 0.8873


결과를 이미지로 저장

In [92]:
def mkdir(path):
    if os.path.exists(path) is False:
        os.mkdir(path)

mkdir(RESULT_SAVE_PATH)
mkdir(RESULT_SAVE_PATH + '/TP')
mkdir(RESULT_SAVE_PATH + '/TN')
mkdir(RESULT_SAVE_PATH + '/FP')
mkdir(RESULT_SAVE_PATH + '/FN')

index = 0
for imgs, labels in ds_valid:
    preds = model.predict(imgs)
    for idx in range(imgs.shape[0]):
        gt = labels[idx].numpy()
        y = preds[idx]
        
        if gt == 1 and y > 0.5:
            path = RESULT_SAVE_PATH + '/TP'
        elif gt == 1 and y <= 0.5:
            path = RESULT_SAVE_PATH + '/FN'
        elif gt == 0 and y > 0.5:
            path = RESULT_SAVE_PATH + '/FP'
        else:
            path = RESULT_SAVE_PATH + '/TN'
            
        cv2.imwrite(path + '/%.4f_%04d.png' % (y, index), imgs[idx].numpy() * 255)
        index +=1